In [12]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("Crop_recommendation.csv")

# Features and target
X = df.drop("label", axis=1)
y = df["label"]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Optional: Evaluate
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))

Model Accuracy: 0.9931818181818182


In [25]:
# ✅ Imports
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import requests
import os
from dotenv import load_dotenv
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from langdetect import detect

# ✅ Load environment variables
load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")
# ✅ NER Setup
ner_model_name = "Davlan/bert-base-multilingual-cased-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=tokenizer, grouped_entities=True)

# ✅ WSD Model and Sense Inventory
wsd_model = SentenceTransformer('all-MiniLM-L6-v2')
sense_inventory = {
    "plant_1": "a living organism like a crop or tree",
    "plant_2": "an industrial facility such as a pesticide plant",
    "spray_1": "the act of spraying liquid on crops",
    "spray_2": "a physical pesticide product in a bottle"
}

def disambiguate_word(context_sentence, word, sense_inventory):
    senses = [k for k in sense_inventory if k.startswith(word)]
    if not senses:
        return None

    context_embedding = wsd_model.encode(context_sentence, convert_to_tensor=True)
    best_sense, best_score = None, -1
    for sense_key in senses:
        gloss = sense_inventory[sense_key]
        gloss_embedding = wsd_model.encode(gloss, convert_to_tensor=True)
        score = util.pytorch_cos_sim(context_embedding, gloss_embedding).item()
        if score > best_score:
            best_score = score
            best_sense = sense_key
    return best_sense

# ✅ Extract location from query
def extract_location(text):
    entities = ner_pipeline(text)
    for ent in entities:
        if ent["entity_group"] == "LOC":
            return ent["word"]
    return None

# ✅ Weather + Crop Suggestion Logic
# def get_weather_desc(location, api_key):
#    try:
#         url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={API_KEY}&units=metric"
#         response = requests.get(url)
#         #print("API Response:", response.json())  # Debugging print
#         data = response.json()
#         if data.get("cod") != 200:
#             return None
#         return data['weather'][0]['description']
#    except Exception as e:
#         print(f"Error: {e}")  # Print error if any
#         return None

def get_weather_data(location, api_key):
    try:
        # Construct the API URL
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"  # 'metric' for Celsius
        response = requests.get(url)
        data = response.json()
        
        if data.get("cod") != 200:
            return None, None  # Return None if API call is unsuccessful
        
        # Extract weather description and temperature
        weather_desc = data['weather'][0]['description']
        temperature = data['main']['temp']  # Temperature in Celsius
        
        return weather_desc, temperature
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# def suggest_crops(weather_desc, month, soil_type="loamy"):
#     crop_db = {
#         "loamy": {
#             "summer": ["maize", "cotton", "groundnut"],
#             "monsoon": ["rice", "millets", "soybean"],
#             "winter": ["wheat", "mustard", "gram"]
#         },
#         "clay": {
#             "summer": ["sugarcane"],
#             "monsoon": ["paddy", "jute"],
#             "winter": ["barley", "wheat"]
#         }
#     }
#     if "rain" in weather_desc:
#         season = "monsoon"
#     elif month in ["March", "April", "May", "June"]:
#         season = "summer"
#     else:
#         season = "winter"
#     return crop_db.get(soil_type, {}).get(season, ["No crop data available"])

# ✅ Main Chatbot Function
def get_crop_recommendation(query):
    ambiguous_words = ["plant", "spray"]
    disambigs = {}
    for word in ambiguous_words:
        if word in query.lower():
            sense = disambiguate_word(query, word, sense_inventory)
            if sense:
                disambigs[word] = sense

    # Handle wrong sense
    if "plant" in disambigs and disambigs["plant"] == "plant_2":
        return "Did you mean a factory? For crop suggestions, please clarify."

    location = extract_location(query)
    if not location:
        return "Please include your location in the query."

    # Get weather data (description and temperature)
    weather_desc, temperature = get_weather_data(location, API_KEY)
    
    if not weather_desc:
        return f"Couldn't get weather info for {location}."

    # Use only the weather description for crop suggestion
    current_month = datetime.now().strftime("%B")
    crops = predict_crop(location)
    
    return f"🌾 Based on {weather_desc} in {location}, recommended crops are: {', '.join(crops)}"

location = "Chennai"  # You can change this to your preferred location
weather_desc, temperature = get_weather_data(location, API_KEY)

if weather_desc and temperature:
    print(f"Weather in {location}: {weather_desc}")
    print(f"Temperature: {temperature}°C")
else:
    print(f"Couldn't get weather info for {location}.")

def predict_crop(city):
    weather_desc,temp = get_weather_data(city, API_KEY)
    if temp is None:
        return f"Couldn't fetch weather info for {city}."

    # Collect other inputs from the user
    # N = float(input("Enter Nitrogen (N) value: "))
    # P = float(input("Enter Phosphorus (P) value: "))
    # K = float(input("Enter Potassium (K) value: "))
    # ph = float(input("Enter soil pH value: "))
    # rainfall = float(input("Enter expected rainfall (mm): "))

    # Create input array
    #input_data = [[N, P, K, temp, humidity, ph, rainfall]]
    avg_N = 90
    avg_P = 42
    avg_K = 43
    avg_ph = 6.5
    avg_rainfall = 100
    humidity = 68

    # Create input with only city-based temp/humidity + defaults
    input_data = [[avg_N, avg_P, avg_K, temp, humidity, avg_ph, avg_rainfall]]
    #input_data = [[ temp]]

    # Predict crop
    crop_encoded = model.predict(input_data)[0]
    crop_label = label_encoder.inverse_transform([crop_encoded])[0]

    return f"🌱 Recommended crop for {city} is: {crop_label}"

city = input("Enter your city: ")
print(predict_crop(city))
# 🧪 Example
query1 = "What plant should I grow in summer in {city}?"
query2 = "Is there a pesticide plant near my village?"
print("User:", query1)
print("Bot:", get_crop_recommendation(query1))
print("\nUser:", query2)
print("Bot:", get_crop_recommendation(query2))

city = input("Enter your city: ")
print(predict_crop(city))


Device set to use cpu
c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


Weather in Chennai: scattered clouds
Temperature: 30.47°C


c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


🌱 Recommended crop for Chennai is: coffee
User: What plant should I grow in summer in {city}?
Bot: Please include your location in the query.

User: Is there a pesticide plant near my village?
Bot: Did you mean a factory? For crop suggestions, please clarify.
Couldn't fetch weather info for .
